In [1]:
SUBTASKS = [2, 3]

In [2]:
from collections import defaultdict
from IPython.display import display
from helper import *
from evaluate import *
import pandas as pd

In [3]:
df_subtask2_train_split = get_performance_tabular(
    "cF1", 2, strategy="train_split")
df_subtask3_train_split = get_performance_tabular(
    "cF1", 3, strategy="train_split")

df_subtasks_train_split = {
    3: df_subtask3_train_split,
    2: df_subtask2_train_split
}

display(df_subtask2_train_split)
display(df_subtask3_train_split)

,Language,Domain,no_sc_no_guided_27b,sc_5_27b,sc_10_27b,sc_15_27b
0,English,Restaurant,0.641013,0.646060,0.657267,0.655753
1,English,Laptop,0.634052,0.638545,0.640785,0.639545
2,Japanese,Hotel,0.540414,0.544074,0.544213,0.550660
3,Russian,Restaurant,0.589030,0.599728,0.601384,0.601334
4,Tatar,Restaurant,0.491330,0.496435,0.507477,0.505180
5,Ukrainian,Restaurant,0.571655,0.579969,0.581387,0.584901
6,Chinese,Restaurant,0.728749,0.731102,0.732214,0.733347
7,Chinese,Laptop,NaN,NaN,NaN,NaN
8,AVG,,0.599463,0.605130,0.609247,0.610103


,Language,Domain,no_sc_no_guided_27b,sc_5_27b,sc_10_27b,sc_15_27b
0,English,Restaurant,None,None,None,None
1,English,Laptop,None,None,None,None
2,Japanese,Hotel,None,None,None,None
3,Russian,Restaurant,None,None,None,None
4,Tatar,Restaurant,None,None,None,None
5,Ukrainian,Restaurant,None,None,None,None
6,Chinese,Restaurant,None,None,None,None
7,Chinese,Laptop,None,None,None,None
8,AVG,,NaN,NaN,NaN,NaN


In [4]:
results_subtask2 = run_significance_tests(df_subtask2_train_split, 2)
results_subtask3 = run_significance_tests(df_subtask3_train_split, 3)


SIGNIFICANCE TESTS - Subtask 2

--- Shapiro-Wilk Normalitätstests ---
no_sc_no_guided_27b: W=0.9819, p=0.9683 (normal)
sc_5_27b: W=0.9858, p=0.9828 (normal)
sc_10_27b: W=0.9859, p=0.9832 (normal)
sc_15_27b: W=0.9890, p=0.9911 (normal)

Alle normalverteilt: True

--- Omnibus-Test (Unterschied zwischen allen 6 Gruppen) ---
ANOVA: Statistik=0.0288, p=0.993238
Signifikanter Unterschied zwischen Gruppen: Nein

--- Paarweise Vergleiche (Bonferroni-Holm) ---


,Vergleich,p,p adj.,adj. α,Signifikant
0,no_sc_no_guided_27b vs sc_15_27b,0.000457,0.002744,0.008333,Ja
1,no_sc_no_guided_27b vs sc_5_27b,0.001967,0.009833,0.010000,Ja
2,no_sc_no_guided_27b vs sc_10_27b,0.002981,0.011922,0.012500,Ja
3,sc_5_27b vs sc_15_27b,0.009458,0.028373,0.016667,Ja
4,sc_5_27b vs sc_10_27b,0.064973,0.129947,0.025000,Nein
5,sc_10_27b vs sc_15_27b,0.497911,0.497911,0.050000,Nein



SIGNIFICANCE TESTS - Subtask 3

--- Shapiro-Wilk Normalitätstests ---

Alle normalverteilt: True

--- Omnibus-Test (Unterschied zwischen allen 6 Gruppen) ---


TypeError: At least two samples are required; got 0.

In [ ]:
display(format_table_parameter_tuning_for_latex(df_subtask2_train_split))
display(format_table_parameter_tuning_for_latex(df_subtask3_train_split))

,Language,Domain,no_sc_no_guided_27b,sc_5_27b,sc_10_27b,sc_15_27b
0,English,Restaurant,64.10,64.61,\textbf{65.73},65.58
1,English,Laptop,63.41,63.85,\textbf{64.08},63.95
2,Japanese,Hotel,54.04,54.41,54.42,\textbf{55.07}
3,Russian,Restaurant,58.90,59.97,\textbf{60.14},60.13
4,Tatar,Restaurant,49.13,49.64,\textbf{50.75},50.52
5,Ukrainian,Restaurant,57.17,58.00,58.14,\textbf{58.49}
6,Chinese,Restaurant,72.87,73.11,73.22,\textbf{73.33}
7,Chinese,Laptop,N/A,N/A,N/A,N/A
8,AVG,,59.95,60.51,60.92,\textbf{61.01}


,Language,Domain,no_sc_no_guided_27b,sc_5_27b,sc_10_27b,sc_15_27b
0,English,Restaurant,N/A,N/A,N/A,N/A
1,English,Laptop,N/A,N/A,N/A,N/A
2,Japanese,Hotel,N/A,N/A,N/A,N/A
3,Russian,Restaurant,N/A,N/A,N/A,N/A
4,Tatar,Restaurant,N/A,N/A,N/A,N/A
5,Ukrainian,Restaurant,N/A,N/A,N/A,N/A
6,Chinese,Restaurant,N/A,N/A,N/A,N/A
7,Chinese,Laptop,N/A,N/A,N/A,N/A
8,AVG,,N/A,N/A,N/A,N/A


In [ ]:
# get df_subtask2_train_split values as 1d list
# get df_subtask2_train_split values as 1d list (ohne Language und Domain)
df_values_subtask2 = format_table_parameter_tuning_for_latex(df_subtask2_train_split).drop(columns=["Language", "Domain"]).values.flatten().tolist()
df_values_subtask3 = format_table_parameter_tuning_for_latex(df_subtask3_train_split).drop(columns=["Language", "Domain"]).values.flatten().tolist()

with open(os.path.join("plots", "parameter_optimization_subtask2.txt"), "w") as f:
    f.write(get_tabular_parameter_optimization(df_values_subtask2))

with open(os.path.join("plots", "parameter_optimization_subtask3.txt"), "w") as f:
    f.write(get_tabular_parameter_optimization(df_values_subtask3))

## Export Predictions in Valid Format

In [ ]:
strategy_export = "pred_dev"

In [ ]:
for subtask in SUBTASKS:
    for language, domain in VALID_LANGUAGES_DOMAINS:
        try:
            # best_strategy = get_key_of_best_strategy(
            #     language, domain, df_subtasks_train_split[subtask])
            best_strategy = "sc_15_27b"
            
            num_sc_bs = int(best_strategy.split("_")[1])
            llm_params = best_strategy.split("_")[2]

            predictions = get_performance(language, domain, subtask, strategy_export, llm=f"unsloth/gemma-3-{llm_params}-it-bnb-4bit", num_preds_sc=num_sc_bs)[1]["sc_no_guided"]
            output_dir = f"exported_predictions/subtask_{subtask}/pred_{language}_{domain}.jsonl"
            os.makedirs(os.path.dirname(output_dir), exist_ok=True)
            with open(output_dir, "w", encoding="utf-8") as f:
                for pred in predictions:
                    f.write(json.dumps(pred, ensure_ascii=False) + "\n")
        except Exception as e:
            print(f"Error processing Subtask {subtask} - Language: {language}, Domain: {domain}: {e}")
            continue
        print(
            f"Subtask {subtask} - Language: {language}, Domain: {domain} => Best Strategy: {best_strategy}")

Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Subtask 2 - Language: eng, Domain: restaurant => Best Strategy: sc_15_27b
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Subtask 2 - Language: eng, Domain: laptop => Best Strategy: sc_15_27b
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot perform evaluation.
Error: Failed to load one or both data files. Cannot 

## Create Tables Dataset Statistics

In [ ]:
import pandas as pd

ROWS = ["train", "dev", "test", "test_cross_validation"]
DOMAIN_ORDER = ["restaurant", "laptop", "hotel"]  # Definiere die gewünschte Reihenfolge
SUBTASK_ORDER = [2, 3]  # Erst Subtask 2, dann Subtask 3

records_dataset_statistics = []

for subtask in SUBTASKS:
    for language, domain in VALID_LANGUAGES_DOMAINS:

        # train + dev
        for split in ["train", "dev"]:
            count = len(get_dataset(subtask, language, domain, split=split))
            records_dataset_statistics.append((split, domain, subtask, language, count))

        # test (optional)
        try:
            count_test = len(get_dataset(subtask, language, domain, split="test"))
            records_dataset_statistics.append(("test", domain, subtask, language, count_test))
        except:
            pass

        # test_cross_validation (als ganze Zahl)
        train_size = len(get_dataset(subtask, language, domain, split="train"))
        records_dataset_statistics.append((
            "test_cross_validation",
            domain,
            subtask,
            language,
            str(int(train_size * 0.2))
        ))

df_dataset_statistics = pd.DataFrame(
    records_dataset_statistics,
    columns=["split", "domain", "subtask", "language", "count"]
)

# Konvertiere domain zu Categorical mit gewünschter Reihenfolge
df_dataset_statistics["domain"] = pd.Categorical(
    df_dataset_statistics["domain"], 
    categories=DOMAIN_ORDER, 
    ordered=True
)

df_dataset_statistics = (
    df_dataset_statistics
        .pivot(index=["split", "domain"],
               columns=["subtask", "language"],
               values="count")
        .sort_index(level=["split", "domain"], key=lambda x: x.map({s: i for i, s in enumerate(ROWS)} if x.name == "split" else {d: i for i, d in enumerate(DOMAIN_ORDER)}))
)

# Spalten nach gewünschter Subtask-Reihenfolge sortieren (erst 2, dann 3)
df_dataset_statistics = df_dataset_statistics.reindex(
    columns=sorted(df_dataset_statistics.columns, key=lambda x: (SUBTASK_ORDER.index(x[0]), x[1]))
)

df_dataset_statistics = df_dataset_statistics.applymap(
    lambda x: f"{int(x):,}" if pd.notna(x) else "-"
)

# get values from left to right from top to bottom as 1D list
values_list_dataset_statistics = df_dataset_statistics.values.flatten().tolist()
df_dataset_statistics

/tmp/ipykernel_2090912/2424028441.py:59: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_dataset_statistics = df_dataset_statistics.applymap(


subtask                               2                                     \
language                            eng    jpn    rus    tat    ukr    zho   
split                 domain                                                 
train                 restaurant  2,284      -  1,240  1,240  1,240  6,050   
                      laptop      4,076      -      -      -      -  3,490   
                      hotel           -  1,600      -      -      -      -   
dev                   restaurant    200      -     48     48     48    300   
                      laptop        200      -      -      -      -    300   
                      hotel           -    200      -      -      -      -   
test_cross_validation restaurant    456      -    248    248    248  1,210   
                      laptop        815      -      -      -      -    698   
                      hotel           -    320      -      -      -      -   

subtask                               3                                     
language                            eng    jpn    rus    tat    ukr    zho  
split                 domain                                                
train                 restaurant  2,284      -  1,240  1,240  1,240  6,050  
                      laptop      4,076      -      -      -      -  3,490  
                      hotel           -  1,600      -      -      -      -  
dev                   restaurant    200      -     48     48     48    300  
                      laptop        200      -      -      -      -    300  
                      hotel           -    200      -      -      -      -  
test_cross_validation restaurant    456      -    248    248    248  1,210  
                      laptop        815      -      -      -      -    698  
                      hotel           -    320      -      -      -      -

In [ ]:
with open("plots/muster/dataset.txt", "r", encoding="utf-8") as f:
    dataset_muster = f.read()

# gehe von xxxx zu xxxx und trage die Werte ein
for value in values_list_dataset_statistics:
    dataset_muster = dataset_muster.replace("xxxx", value, 1)

with open("plots/dataset_statistics.txt", "w", encoding="utf-8") as f:
    f.write(dataset_muster)